# **Applied Data Science in Finance**

---

In this notebook, we would like to apply data science on quantitative financial topics. We will apply neural network method on our option pricing tools. The notebook will cover from the option pricing tool to complicated quantitative models such as Black Scholes and Heston model. 

In this notebook, we use call option as an example to generate all the results.



In [ ]:
import scipy
from scipy import stats
import numpy as np
import pandas as pd
import scipy.stats as ss
from numpy import sqrt, exp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

## 1.1 Analytical option pricing tool

In [ ]:
# Generate the cumulative distribution function 
cdf = stats.norm(0, 1).cdf
"""
  K is the strike price
  S asset price
  T Maturity 
  r risk free rate 
  q dividend rate
  σ volatility

"""

# Define D1
def d1(S, K, T, r, σ):
    return (np.log(S / K) + ((r) + 0.5 * σ ** 2) * T) / (σ * np.sqrt(T))

#Define D2
def d2(d1,T,σ):
    return d1- σ * np.sqrt(T)

# Calls
def call(S, K, T, r,d1,d2):
    return S *np.exp(r*T)* cdf(d1) - K * np.exp(-r * T) * cdf(d2)

#Puts
def put(S, K, T, r,d1,d2):
    return np.exp(-r * T) * K * cdf(-d2) - S *np.exp(r*T)* cdf(-d1)

In [ ]:
# Trial with the above defined formula
S,K,T,r,q,σ= 100, 100, 1, 0.05, 0.0, 0.3
d1 = d1(S, K, T, r, σ)
d2 = d2(d1,T,σ)
C = call(S, K, T, r,d1,d2)
P = put(S, K, T, r,d1,d2)
print(C);print(P)

17.431861836422698
7.427694648891695


In [ ]:
# Create an input dataset
spot_prices=[v for v in range(5,100,5)] # Generate a list starting with 5 and ending with 100 with an interval of 5
strike_prices=[u for u in range(5,90,10)] # Generate a list starting with 5 and ending with 90 with an interval of 10
interest_rates=[0.01,0.015,0.02,0.025]
volatilities=[0.05,0.1,0.15,0.2,0.25,0.3]
expiry_times=[7,14,21,28,35,42,49]

From this step, we would like to apply the input dataset we create above in our analytical option tool, in order to generate a full dataset with the input and output. 

Then, we split the complete dataset into training and testing set. As default, the testing dataset will remain at 20% of the whole dataset.

In [ ]:
# Generate whole dataset and split it into training and testing
call_option=[]
for σ in volatilities:
    for r in interest_rates:
        for T in expiry_times:
            for K in strike_prices:
                for S in spot_prices:
                    price=call(S, K, T, r,d1,d2)
                
                    call_option.append({
                        'volatility':σ,
                        'interest_rate':r,
                        'expiry_time':T,
                        'initial_price':S,
                        'strike_price':K,
                        'price':price
                    })
df=pd.DataFrame.from_dict(call_option)
df.head()
x=df[['volatility','interest_rate','expiry_time','initial_price','strike_price']].values
y=df['price'].values
x_train, x_val, y_train, y_val=train_test_split(x,y,test_size=20)

## 1.2 Analytical option pricing tool with neural network training
After creating the analytical option pricing tool, we would like to apply neural network model to train our dataset. 

In [ ]:
# 2 hidden layer to predict an output of 1 dimension (price) with 5 dimension input
def create_network(input_dim, output_dim, layer1_dim, layer2_dim):
    inputs=tf.keras.Input(shape=(input_dim,))
    encoded1=tf.keras.layers.Dense(layer1_dim,activation='relu')(inputs)
    encoded2=tf.keras.layers.Dense(layer2_dim,activation='relu')(encoded1)
    output=tf.keras.layers.Dense(output_dim)(encoded2)
    model=tf.keras.Model(inputs,output)
    return model

We choose 200 as epochs, as we observe the val_loss starts to remain stable after 20 training. Therefore, we keep 200 as a threshold for the epochs. In terms of shuffle, we put it into "False" to keep a lower val_loss.

In [ ]:
# Integrate inputs and train dataset
input_dim = x_train.shape[1]
output_dim = 1
model = create_network(input_dim,output_dim, 200, 200)
model.compile(optimizer='adam',loss='mean_squared_error')
training=model.fit(x_train,y_train,epochs=200,batch_size=32,shuffle=False,validation_data=(x_val,y_val))

Epoch 1/200
898/898 [==============================] - 2s 2ms/step - loss: 411.3624 - val_loss: 124.2580
Epoch 2/200
898/898 [==============================] - 1s 1ms/step - loss: 265.4288 - val_loss: 130.0351
Epoch 3/200
898/898 [==============================] - 1s 1ms/step - loss: 250.5447 - val_loss: 136.9921
Epoch 4/200
898/898 [==============================] - 1s 2ms/step - loss: 241.2757 - val_loss: 145.1490
Epoch 5/200
898/898 [==============================] - 1s 1ms/step - loss: 234.9283 - val_loss: 150.9789
Epoch 6/200
898/898 [==============================] - 1s 1ms/step - loss: 229.5960 - val_loss: 149.6429
Epoch 7/200
898/898 [==============================] - 1s 1ms/step - loss: 224.1866 - val_loss: 162.4435
Epoch 8/200
898/898 [==============================] - 1s 1ms/step - loss: 218.4896 - val_loss: 160.1441
Epoch 9/200
898/898 [==============================] - 1s 1ms/step - loss: 212.7361 - val_loss: 161.3345
Epoch 10/200
898/898 [==============================] -

In [ ]:
data_test=[]
num=1000

for n in range(num):
  σ = np.random.uniform(low=volatilities[0],high=volatilities[-1])
  r=np.random.uniform(low=interest_rates[0],high=interest_rates[-1])
  T=np.random.uniform(low=expiry_times[0],high=expiry_times[-1])
  S=np.random.uniform(low=spot_prices[0],high=spot_prices[-1])
  K=np.random.uniform(low=strike_prices[0],high=strike_prices[-1])

  price=call(S, K, T, r,d1,d2)
                
  data_test.append({
        'volatility':σ,
        'interest_rate':r,
        'expiry_time':T,
        'initial_price':S,
        'strike_price':K,
        'price':price
      })
  
df_test=pd.DataFrame.from_dict(data_test)
df_test.head()

X_test=df_test[['volatility','interest_rate','expiry_time','initial_price','strike_price']].values
y_test=df_test['price'].values

loss = model.evaluate(X_test, y_test, verbose=0)

print("average absolute error on test data is {}".format(np.sqrt(loss)))

average absolute error on test data is 0.915320451883592


## 2. Black Scholes Option Pricing via Monte Carlo paths simulation
We apply the neural network on Black Scholes model with Monte Carlo paths simulation. To have larger dataset, we choose to simulate 20 000 paths.

In [ ]:
# Function MonteCarlo Pricing
def priceEuropeanCallMC(S0,K,r,T,sigma,M,paths):
    
    # generate M samples from N(0,1)
    X = np.random.randn(paths)
    
    # simulate M trajectories in one step
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * X)

    # define payoff
    payoff = np.where(ST < K, 0, ST - K) # acts as max(ST-K, 0) 
    
    # MC estimate
    discountFactor = np.exp(-r*T)
    
    price_MC = discountFactor*np.mean(payoff)
    return price_MC


In [ ]:
# Produce 20 000 Prices Sample using MC Simulation

paths = 20000

data = []

for n in range(nb_samples): 
    
    sigma = np.random.uniform(low=volatilities[0], high=volatilities[-1])
    
    r = np.random.uniform(low=interest_rates[0], high=interest_rates[-1])
    
    T = np.random.uniform(low=expiry_times[0], high=expiry_times[-1])
    
    S0 = np.random.uniform(low=spot_prices[0], high=spot_prices[-1])
    
    K = np.random.uniform(low=(1-0.2)*S0, high=(1+0.2)*S0)
    
    M = 2000

    price = priceEuropeanCallMC(S0, K, r, T, sigma, M, paths)

    data.append({
        'volatility': sigma,
        'interest_rate': r,
        'expiry_time': T,
        'spot_price': S0,
        'strike_price': K,
        'price': price
    })

df_MC = pd.DataFrame.from_dict(data)

In [ ]:
# Generate x_train and y_train for Model Training
x=df_MC[['volatility','interest_rate','expiry_time','spot_price','strike_price']].values
y=df_MC['price'].values
x_train, x_val, y_train, y_val=train_test_split(x,y,test_size=20)

The neural network model will not be changed from the above one. As a result, we don't define a new model in this case.

In [ ]:
# Training model with MC Simulation (20 000 Option Prices Simulation with MC Sumulating 20 000 paths)
input_dim = x_train.shape[1]
output_dim = 1
model = create_network(input_dim,output_dim, 200, 200)
model.compile(optimizer='adam',loss='mean_squared_error')
training=model.fit(x_train,y_train,epochs=200,batch_size=32,shuffle=False,validation_data=(x_val,y_val))

Epoch 1/200
625/625 [==============================] - 1s 2ms/step - loss: 86.2502 - val_loss: 17.1830
Epoch 2/200
625/625 [==============================] - 1s 2ms/step - loss: 32.4992 - val_loss: 14.9708
Epoch 3/200
625/625 [==============================] - 1s 2ms/step - loss: 24.7940 - val_loss: 8.3650
Epoch 4/200
625/625 [==============================] - 1s 2ms/step - loss: 18.1346 - val_loss: 8.8246
Epoch 5/200
625/625 [==============================] - 1s 2ms/step - loss: 12.8777 - val_loss: 8.9332
Epoch 6/200
625/625 [==============================] - 1s 2ms/step - loss: 10.5529 - val_loss: 9.1038
Epoch 7/200
625/625 [==============================] - 1s 2ms/step - loss: 9.4780 - val_loss: 8.9946
Epoch 8/200
625/625 [==============================] - 1s 2ms/step - loss: 8.7821 - val_loss: 8.8584
Epoch 9/200
625/625 [==============================] - 1s 2ms/step - loss: 8.2847 - val_loss: 8.2438
Epoch 10/200
625/625 [==============================] - 1s 2ms/step - loss: 7.9944 

In [ ]:
# Testing Model after training with MC Silumations of 20 000 Paths
data_test = []
paths = 20000

for n in range(num):
    
    sigma = np.random.uniform(low=volatilities[0], high=volatilities[-1])
    
    r = np.random.uniform(low=interest_rates[0], high=interest_rates[-1])
    
    T = np.random.uniform(low=expiry_times[0], high=expiry_times[-1])
    
    S0 = np.random.uniform(low=spot_prices[0], high=spot_prices[-1])
    
    K = np.random.uniform(low=(1-0.2)*S0, high=(1+0.2)*S0)
    
    M = 2000

    price = priceEuropeanCallMC(S0, K, r, T, sigma, M, paths)

    data.append({
        'volatility': sigma,
        'interest_rate': r,
        'expiry_time': T,
        'spot_price': S0,
        'strike_price': K,
        'price': price
    })


x_test_MC = df_MC[['volatility','interest_rate','expiry_time','spot_price','strike_price']].values
y_test_MC = df_MC['price'].values

loss = model.evaluate(x_test_MC, y_test_MC, verbose=0)

print("average absolute error on test data is {}".format(np.sqrt(loss)))

average absolute error on test data is 2.272863046751997


## 3. Heston Model Pricer

In this part, we will generate option price with Heston models via Monte Carlo simulation. Then, we will utilize neural network to train and test the model.

In [ ]:
# Define Heston model formula
def mc_heston(option_type,S0,K,T,V0,theta,kappa,zeta,rho,r,M):
    """
    option_type:    'p' put option 'c' call option
    S0:              the spot price of underlying stock
    K:              the strike price
    T:              the maturity of options
    initial_var:    the initial value of variance
    theta:          the long term average of price variance
    kappa:          the mean reversion rate for the variance
    zeta:           the volatility of volatility(the variance of the variance of stock price)
    rho:            the correlation between the standard normal random variables W1 and W2
    r:              the risk free rate
    paths:          the number of repeat for monte carlo simulation
    M:              the number of steps in each simulation
    """
    dt = float(T)/float(M)
    payoff = 0
    S =[None] * M # Size of the series
    S[0]=S0
    V =[None] * M # Size of the series
    V[0]=V0
    
    for t in range(1,M):
      # generate Monte Carlo paths
      w1 = np.random.normal(0, 1, M) 
      w2 = rho*w1+sqrt(1-rho**2)*np.random.normal(0, 1, M)
      # volatility formula
      V[t] = V[t-1] + kappa * (theta - V[t-1]) * dt + zeta * np.sqrt(V[t-1] * dt) * w2
      # price formula
      S[t] = S[t-1] * np.exp(np.sqrt(V[t-1] * dt) * w1 - V[t-1] * dt / 2)
        
      # option price with different types
      if option_type == 'c':
          payoff = np.where(S[t] < K, 0, S[t] - K) # acts as max(St-K, 0)
      elif option_type == 'p':
          payoff = np.where(S[t] < K, K - S[t], 0) # acts as max(K-St, 0)

    return payoff

In [ ]:
# Heston option pricer test
heston = {}
option_type = 'c'
S0 = 105
K = 100
T = 1
V0 = 0.01
theta = 0.01
kappa = 2
zeta = 0.1
rho = 0
r = 0.5
M = 1000

heston = {'Option Price' : mc_heston(option_type,S0,K,T,V0,theta,kappa,zeta,rho,r,M)}

df_mc_heston=pd.DataFrame.from_dict(heston)

MC_Heston_Simulation = np.exp(-r*T) * df_mc_heston["Option Price"].mean()

print(MC_Heston_Simulation)


4.208751065798564


In [ ]:
# Build input dataset
spot_prices=[v for v in range(5,100,5)] #generate a list starting with 5 and ending with 100 with an interval of 5
strike_prices=[u for u in range(5,90,10)]
interest_rates=[0.01,0.015,0.02,0.025]
volatilities=[0.05,0.1,0.15,0.2,0.25,0.3]
expiry_times=[7,14,21,28,35,42,49]
long_term_variances=[0.01,0.02,0.03,0.04,0.05]
mean_reversion=[2,4,6,8,10]
vol_of_vol=[0.1,0.2,0.3,0.4,0.5]
corr=[-0.2,-0.1,0,0.1,0.2]

In [ ]:
# Produce 1000 Prices Sample using MC Simulation

data = []
paths = 1000

for n in range(paths): 

    S0 = np.random.uniform(low=spot_prices[0], high=spot_prices[-1])

    K = np.random.uniform(low=(1-0.2)*S0, high=(1+0.2)*S0)

    T = np.random.uniform(low=expiry_times[0], high=expiry_times[-1])

    V0 = np.random.uniform(low=volatilities[0], high=volatilities[-1])

    theta = np.random.uniform(low=long_term_variances[0], high=long_term_variances[-1])

    kappa = np.random.uniform(low=mean_reversion[0], high=mean_reversion[-1])

    zeta = np.random.uniform(low=vol_of_vol[0], high=vol_of_vol[-1])

    rho = np.random.uniform(low=corr[0], high=corr[-1])
    
    r = np.random.uniform(low=interest_rates[0], high=interest_rates[-1])

    M = 1000

    price=mc_heston('c',S0,K,T,V0,theta,kappa,zeta,rho,r,M).mean()


    data.append({
        'volatility': V0,
        'interest_rate': r,
        'expiry_time': T,
        'spot_price': S0,
        'strike_price': K,
        'long_term_variance': theta,
        'mean_reversion': kappa,
        'vol_of_vol': zeta,
        'corr': rho,
        'price': price
    })

df_heston = pd.DataFrame.from_dict(data)
df_heston = df_heston.fillna(0)

x=df_heston[['volatility','interest_rate','expiry_time','spot_price','strike_price','long_term_variance',
             'mean_reversion','vol_of_vol','corr']].values
y=df_heston['price']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
# Generate x_train and y_train for Model Training
x=df_heston[['volatility','interest_rate','expiry_time','spot_price','strike_price','long_term_variance',
             'mean_reversion','vol_of_vol','corr']].values
y=df_heston['price'].values
x_train, x_val, y_train, y_val=train_test_split(x,y,test_size=20)

As indicated, we need to build a new neural network model with 3 layers. Although Heston model adds volatility in its calculation, the output dimension is still 1, which is the price.

In [ ]:
# 3 hidden layer to predict an output of 1 dimension (price and volatility) with 9 dimension input
def create_network_heston(input_dim, output_dim, layer1_dim, layer2_dim, layer3_dim):
    inputs=tf.keras.Input(shape=(input_dim,))
    encoded1=tf.keras.layers.Dense(layer1_dim,activation='relu')(inputs)
    encoded2=tf.keras.layers.Dense(layer2_dim,activation='relu')(encoded1)
    encoded3=tf.keras.layers.Dense(layer3_dim,activation='relu')(encoded2)
    output=tf.keras.layers.Dense(output_dim)(encoded3)
    model_heston=tf.keras.Model(inputs,output)
    return model_heston

In [ ]:
# Training model with MC Simulation (1000 Option Prices Simulation with MC Sumulating 200 paths)
input_dim = x_train.shape[1]
output_dim = 1
model_heston = create_network_heston(input_dim,output_dim, 200, 150,150)
model_heston.compile(optimizer='adam',loss='mean_squared_error')
training=model_heston.fit(x_train,y_train,epochs=200,batch_size=30,shuffle=False,validation_data=(x_val,y_val))

Epoch 1/200
33/33 [==============================] - 1s 7ms/step - loss: 91.0203 - val_loss: 66.5337
Epoch 2/200
33/33 [==============================] - 0s 3ms/step - loss: 78.8994 - val_loss: 70.8798
Epoch 3/200
33/33 [==============================] - 0s 3ms/step - loss: 79.2729 - val_loss: 69.0126
Epoch 4/200
33/33 [==============================] - 0s 3ms/step - loss: 78.7057 - val_loss: 68.1400
Epoch 5/200
33/33 [==============================] - 0s 3ms/step - loss: 78.5839 - val_loss: 68.7136
Epoch 6/200
33/33 [==============================] - 0s 3ms/step - loss: 78.4059 - val_loss: 68.6271
Epoch 7/200
33/33 [==============================] - 0s 3ms/step - loss: 78.2061 - val_loss: 68.0955
Epoch 8/200
33/33 [==============================] - 0s 3ms/step - loss: 77.9929 - val_loss: 67.9268
Epoch 9/200
33/33 [==============================] - 0s 3ms/step - loss: 77.6425 - val_loss: 67.3983
Epoch 10/200
33/33 [==============================] - 0s 3ms/step - loss: 77.4703 - val_los

In [ ]:
# Testing Model after training with MC Silumations of 1000 Paths
data = []
paths = 1000

for n in range(paths): 

    S0 = np.random.uniform(low=spot_prices[0], high=spot_prices[-1])

    K = np.random.uniform(low=(1-0.2)*S0, high=(1+0.2)*S0)

    T = np.random.uniform(low=expiry_times[0], high=expiry_times[-1])

    V0 = np.random.uniform(low=volatilities[0], high=volatilities[-1])

    theta = np.random.uniform(low=long_term_variances[0], high=long_term_variances[-1])

    kappa = np.random.uniform(low=mean_reversion[0], high=mean_reversion[-1])

    zeta = np.random.uniform(low=vol_of_vol[0], high=vol_of_vol[-1])

    rho = np.random.uniform(low=corr[0], high=corr[-1])
    
    r = np.random.uniform(low=interest_rates[0], high=interest_rates[-1])

    M = 1000

    price=mc_heston('c',S0,K,T,V0,theta,kappa,zeta,rho,r,M).mean()


    data.append({
        'volatility': V0,
        'interest_rate': r,
        'expiry_time': T,
        'spot_price': S0,
        'strike_price': K,
        'long_term_variance': theta,
        'mean_reversion': kappa,
        'vol_of_vol': zeta,
        'corr': rho,
        'price': price
    })

df_heston = pd.DataFrame.from_dict(data)
df_heston = df_heston.fillna(0)


x_test_heston = df_heston[['volatility','interest_rate','expiry_time','spot_price','strike_price','long_term_variance',
             'mean_reversion','vol_of_vol','corr']].values
y_test_heston = df_heston['price'].values

loss_heston = model_heston.evaluate(x_test_heston, y_test_heston, verbose=0)

print("average absolute error on test data for Heston model is {}".format(np.sqrt(loss_heston)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in sqrt


average absolute error on test data for Heston model is 7.606948747723248


# Conclusion
From the above three different models, we note that the analytical option pricing tool has the lowest MSE result; the Black Scholes model follow tightly and the last one is Heston model. 

However, we cannot conclude that our neural network model doesn't work well on Heston model, since it has 3 layers while the other two has two layers. We have tried the Heston model with a neural network of 2 layers, the result is much better. Therefore, we can see that different layers in neural network system can be vital to the result.

We can conclude that neural network models are really sensitive to their hyperparameters. We should try various hyperparameters to find the most optimised ones. Since the selection process can be long and random, we choose not to show in this notebook.

In addition, we have set up "shuffle" to "False" in all 3 models. As a result, the random possibilities might be biased in our traing set.

